#### Produce cleaning data

In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import geohash_hilbert as ghh

DEVELOPE = False

### Step1 download and read data
Save a smaller sample when developing

#### cells above should be resumed when download original data

## read data

In [2]:
#downloaded from TLC by month
df = pd.read_csv('data/yellow_tripdata_2015-01.csv', index_col=0)
print ("Shape of data\n{}".format(df.shape))
df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Shape of data
(12748986, 18)


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
VendorID,,,,,,,,,,,,,,,,,,
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [3]:
df.reset_index(inplace=True)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


## clean data

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [5]:
df = df.drop(['VendorID', 'trip_distance', 'improvement_surcharge',
         'RateCodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
         'tip_amount', 'tolls_amount', 'total_amount'], axis=1)
df.dropna(inplace=True)
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2015-01-15 19:05:39,2015-01-15 19:23:42,1,-73.993896,40.750111,-73.974785,40.750618
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,-74.001648,40.724243,-73.994415,40.759109
2,2015-01-10 20:33:38,2015-01-10 20:43:41,1,-73.963341,40.802788,-73.951820,40.824413
3,2015-01-10 20:33:39,2015-01-10 20:35:31,1,-74.009087,40.713818,-74.004326,40.719986
4,2015-01-10 20:33:39,2015-01-10 20:52:58,1,-73.971176,40.762428,-74.004181,40.742653


In [6]:
df.shape

(12748986, 7)

#### borders of NY City
city_long_border = (-74.03, -73.75);
city_lat_border = (40.63, 40.85) 

In [7]:
df = df[df['pickup_longitude'] <= -73.75]
df = df[df['pickup_longitude'] >= -74.03]
df = df[df['pickup_latitude'] <= 40.85]
df = df[df['pickup_latitude'] >= 40.63]
df = df[df['dropoff_longitude'] <= -73.75]
df = df[df['dropoff_longitude'] >= -74.03]
df = df[df['dropoff_latitude'] <= 40.85]
df = df[df['dropoff_latitude'] >= 40.63]
df.shape

(12339104, 7)

#### extract all the time features out

In [8]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y/%m/%d %H:%M:%S')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y/%m/%d %H:%M:%S')

In [9]:
df.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
pickup_longitude                float64
pickup_latitude                 float64
dropoff_longitude               float64
dropoff_latitude                float64
dtype: object

In [10]:
df['year'] = df['tpep_pickup_datetime'].dt.year
df['month'] = df['tpep_pickup_datetime'].dt.month
df['hour'] = df['tpep_pickup_datetime'].dt.hour
df['dayofweek'] = df['tpep_pickup_datetime'].dt.weekday_name
df['year_quarter'] = df['tpep_pickup_datetime'].dt.quarter
df['dayofyear'] = df['tpep_pickup_datetime'].dt.dayofyear

In [11]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,hour,dayofweek,year_quarter,dayofyear
0,2015-01-15 19:05:39,2015-01-15 19:23:42,1,-73.993896,40.750111,-73.974785,40.750618,2015,1,19,Thursday,1,15
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,-74.001648,40.724243,-73.994415,40.759109,2015,1,20,Saturday,1,10
2,2015-01-10 20:33:38,2015-01-10 20:43:41,1,-73.963341,40.802788,-73.951820,40.824413,2015,1,20,Saturday,1,10
3,2015-01-10 20:33:39,2015-01-10 20:35:31,1,-74.009087,40.713818,-74.004326,40.719986,2015,1,20,Saturday,1,10
4,2015-01-10 20:33:39,2015-01-10 20:52:58,1,-73.971176,40.762428,-74.004181,40.742653,2015,1,20,Saturday,1,10


In [12]:
df.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'year', 'month', 'hour', 'dayofweek',
       'year_quarter', 'dayofyear'],
      dtype='object')

#### delete trips beyond reasonable time range

In [13]:
df['timerange'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [14]:
df['timerange'] = df['timerange'].dt.seconds

In [15]:
df['timerange']

0           1083
1           1190
2            603
3            112
4           1159
5           1213
6           1492
7            521
8           2276
9            424
10           479
11           585
12           102
13          1763
14           342
15          1581
16           920
17          1358
18           895
19           881
20          1519
21           281
22           314
23           415
24           990
25           504
26           636
27          2605
28           771
29           958
            ... 
12748956     651
12748957    1601
12748958     850
12748959     954
12748960     253
12748961     606
12748962     992
12748963     813
12748964     709
12748965    1134
12748966    1176
12748967     464
12748968     454
12748969     721
12748970     644
12748971     388
12748972     836
12748973     614
12748974     819
12748975     692
12748976     741
12748977    1325
12748978     622
12748979    1530
12748980    1105
12748981     236
12748982     342
12748983     7

In [16]:
df['timerange'].describe()

count    1.233910e+07
mean     7.872938e+02
std      2.225673e+03
min      0.000000e+00
25%      3.660000e+02
50%      5.930000e+02
75%      9.360000e+02
max      8.639900e+04
Name: timerange, dtype: float64

In [17]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,hour,dayofweek,year_quarter,dayofyear,timerange
0,2015-01-15 19:05:39,2015-01-15 19:23:42,1,-73.993896,40.750111,-73.974785,40.750618,2015,1,19,Thursday,1,15,1083
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,-74.001648,40.724243,-73.994415,40.759109,2015,1,20,Saturday,1,10,1190
2,2015-01-10 20:33:38,2015-01-10 20:43:41,1,-73.963341,40.802788,-73.951820,40.824413,2015,1,20,Saturday,1,10,603
3,2015-01-10 20:33:39,2015-01-10 20:35:31,1,-74.009087,40.713818,-74.004326,40.719986,2015,1,20,Saturday,1,10,112
4,2015-01-10 20:33:39,2015-01-10 20:52:58,1,-73.971176,40.762428,-74.004181,40.742653,2015,1,20,Saturday,1,10,1159


In [18]:
df['timerange'].unique()

array([ 1083,  1190,   603, ..., 11445,  6716, 80161])

In [19]:
#12 hours: 12*60*60=43200
df = df[df['timerange'] > 10]
df = df[df['timerange'] < 43200]

In [20]:
#this column is generated for merging with FHV data
df['day_and_hour'] = df['dayofyear'].astype(str) + df['hour'].astype(str)

In [21]:
df.describe()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,hour,year_quarter,dayofyear,timerange
count,1.231601e+07,1.231601e+07,1.231601e+07,1.231601e+07,1.231601e+07,12316008.0,12316008.0,1.231601e+07,12316008.0,1.231601e+07,1.231601e+07
mean,1.687719e+00,-7.397503e+01,4.075120e+01,-7.397452e+01,4.075171e+01,2015.0,1.0,1.358647e+01,1.0,1.603880e+01,7.325861e+02
std,1.344598e+00,3.457108e-02,2.647299e-02,3.071693e-02,2.856598e-02,0.0,0.0,6.389069e+00,0.0,8.776715e+00,6.147616e+02
min,0.000000e+00,-7.402998e+01,4.063002e+01,-7.403000e+01,4.063000e+01,2015.0,1.0,0.000000e+00,1.0,1.000000e+00,1.100000e+01
25%,1.000000e+00,-7.399188e+01,4.073727e+01,-7.399133e+01,4.073626e+01,2015.0,1.0,9.000000e+00,1.0,9.000000e+00,3.660000e+02
50%,1.000000e+00,-7.398196e+01,4.075401e+01,-7.398030e+01,4.075436e+01,2015.0,1.0,1.400000e+01,1.0,1.600000e+01,5.940000e+02
75%,2.000000e+00,-7.396805e+01,4.076792e+01,-7.396440e+01,4.076899e+01,2015.0,1.0,1.900000e+01,1.0,2.300000e+01,9.360000e+02
max,9.000000e+00,-7.375115e+01,4.084999e+01,-7.375000e+01,4.085000e+01,2015.0,1.0,2.300000e+01,1.0,3.100000e+01,4.318600e+04


#### For the holiday:
#### I tried to extract whether certain day is the day before a holiday, which I expect lots of people will rush to airport. However I fail to use the 'Holidays / Holiday Calendars' class in pandas which contains 'USFederalHolidayCalendar'.

# Geohash

In [22]:
df.reset_index(drop=True, inplace=True)

In [23]:
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'year', 'month', 'year_quarter', 'timerange'], axis=1)

In [24]:
#add a column of geohash to dataframe
geo = {}
for i in range(len(df)):
    value = ghh.encode(df.pickup_longitude.iloc[i], df.pickup_latitude.iloc[i], precision=5)
    geo[i] = value
s = pd.Series(geo, name='geohash_pickup')
geo = pd.DataFrame(s)
df = df.merge(geo, left_index=True, right_index=True)
df.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,hour,dayofweek,dayofyear,day_and_hour,geohash_pickup
0,1,-73.993896,40.750111,-73.974785,40.750618,19,Thursday,15,1519,SHG1a
1,1,-74.001648,40.724243,-73.994415,40.759109,20,Saturday,10,1020,SHG2O
2,1,-73.963341,40.802788,-73.951820,40.824413,20,Saturday,10,1020,SHBJ@
3,1,-74.009087,40.713818,-74.004326,40.719986,20,Saturday,10,1020,SHGCP
4,1,-73.971176,40.762428,-74.004181,40.742653,20,Saturday,10,1020,SHG1R


In [25]:
#add a column of geohash to dataframe
geo = {}
for i in range(len(df)):
    value = ghh.encode(df.dropoff_longitude.iloc[i], df.dropoff_latitude.iloc[i], precision=5)
    geo[i] = value
s = pd.Series(geo, name='geohash_dropoff')
geo = pd.DataFrame(s)
df = df.merge(geo, left_index=True, right_index=True)
df.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,hour,dayofweek,dayofyear,day_and_hour,geohash_pickup,geohash_dropoff
0,1,-73.993896,40.750111,-73.974785,40.750618,19,Thursday,15,1519,SHG1a,SHG1X
1,1,-74.001648,40.724243,-73.994415,40.759109,20,Saturday,10,1020,SHG2O,SHG1_
2,1,-73.963341,40.802788,-73.951820,40.824413,20,Saturday,10,1020,SHBJ@,SHBJ_
3,1,-74.009087,40.713818,-74.004326,40.719986,20,Saturday,10,1020,SHGCP,SHGC_
4,1,-73.971176,40.762428,-74.004181,40.742653,20,Saturday,10,1020,SHG1R,SHGDK


In [26]:
df.describe()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,hour,dayofyear
count,1.231601e+07,1.231601e+07,1.231601e+07,1.231601e+07,1.231601e+07,1.231601e+07,1.231601e+07
mean,1.687719e+00,-7.397503e+01,4.075120e+01,-7.397452e+01,4.075171e+01,1.358647e+01,1.603880e+01
std,1.344598e+00,3.457108e-02,2.647299e-02,3.071693e-02,2.856598e-02,6.389069e+00,8.776715e+00
min,0.000000e+00,-7.402998e+01,4.063002e+01,-7.403000e+01,4.063000e+01,0.000000e+00,1.000000e+00
25%,1.000000e+00,-7.399188e+01,4.073727e+01,-7.399133e+01,4.073626e+01,9.000000e+00,9.000000e+00
50%,1.000000e+00,-7.398196e+01,4.075401e+01,-7.398030e+01,4.075436e+01,1.400000e+01,1.600000e+01
75%,2.000000e+00,-7.396805e+01,4.076792e+01,-7.396440e+01,4.076899e+01,1.900000e+01,2.300000e+01
max,9.000000e+00,-7.375115e+01,4.084999e+01,-7.375000e+01,4.085000e+01,2.300000e+01,3.100000e+01


In [27]:
df.dtypes

passenger_count        int64
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
hour                   int64
dayofweek             object
dayofyear              int64
day_and_hour          object
geohash_pickup        object
geohash_dropoff       object
dtype: object

In [28]:
len(df.geohash_dropoff.unique())

992

In [29]:
df.to_csv('clean_taxi_data.csv')